In [1]:
def download_data():
  #download the dataset
  !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
  #unzip the dataset
  !unzip tiny-imagenet-200.zip

download_data()

Output hidden; open in https://colab.research.google.com to view.

In [2]:
#prepare val data classes to file relation
import pandas as pd
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head()

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727
3,val_3.JPEG,n02808440
4,val_4.JPEG,n02808440


In [4]:
!pip install six numpy scipy Pillow matplotlib scikit-image opencv-python imageio Shapely
!pip install imgaug


from imgaug import augmenters as iaa

aug1 = iaa.Sequential([iaa.GaussianBlur(sigma=(0, 2.0)),
                      iaa.Dropout(0.02, name="Dropout"),
                      iaa.AdditiveGaussianNoise(scale=0.01*255, name="MyLittleNoise"),
                      iaa.AdditiveGaussianNoise(loc=32, scale=0.0001*255, name="SomeOtherNoise"),
                      iaa.Affine(translate_px={"x": (-40, 40)}, name="Affine")
                      ])

def other_augmentations(image):
  image = aug1.augment_image(image)
  return image


# Use Augmentaion parameters as required.
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range = 0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=120,
    horizontal_flip=True,
    vertical_flip=True,
    zca_whitening=True, 
    zca_epsilon=1e-06,
    fill_mode = "reflect",
    preprocessing_function=other_augmentations
    )

valid_datagen = ImageDataGenerator(rescale=1./255)

    100% |████████████████████████████████| 17.3MB 2.6MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:334: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [15]:
#initialize train and val data generator
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(64, 64), color_mode='rgb', 
                                                    batch_size=500, class_mode='categorical', shuffle=True, seed=42)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(64,64),
                                                    color_mode='rgb', class_mode='categorical', batch_size=500, shuffle=True, seed=42)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [16]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

from keras.initializers import glorot_uniform


def identity_block(input_tensor, kernel_size, filters, stage, block):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  # for batch normalization layer, we assume
  # the input data is in channel last format
  bn_axis = 3

  filters1, filters2, filters3 = filters

  # main path, note that setting the kernel_initializer seed here is only used
  # for reproducibility, we techniqually don't need it
  x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='valid', name=conv_name_base + '2a')(input_tensor)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='same', name=conv_name_base + '2b')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
  x = layers.Activation('relu')(x)
  
  x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='valid', name=conv_name_base + '2c')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

  # this line is the core component of resnet, the skip connection, i.e.
  # having a shortcut to the main path before the activation, when addition
  # is performed on convolutional layers, the element-wise addition is performed
  # on their feature maps, i.e. channel by channel
  x = layers.add([x, input_tensor])
  x = layers.Activation('relu')(x)
  return x
  
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  # for batch normalization layer, we assume
  # the input data is in channel last format,
  # which is the case if we are using the default
  # keras' backend tensorflow
  bn_axis = 3

  filters1, filters2, filters3 = filters

  # main path, note that setting the kernel_initializer set here is only used
  # for reproducibility, we techniqually don't need it
  x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=strides,
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='valid', name=conv_name_base + '2a')(input_tensor)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
  x = layers.Activation('relu')(x)

  x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='same', name=conv_name_base + '2b')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
  x = layers.Activation('relu')(x)

  
  x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='valid', name=conv_name_base + '2c')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

  # we resize the input so its dimension will match the output dimension
  # of the main path
  shortcut = layers.Conv2D(filters3, kernel_size=(1, 1), strides=strides,
                           kernel_initializer=glorot_uniform(seed=0),
                           padding='valid', name=conv_name_base + '1')(input_tensor)
  shortcut = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut) 

  # this line is the core component of resnet, the skip connection, i.e.
  # having a shortcut to the main path before the activation
  x = layers.add([x, shortcut])
  x = layers.Activation('relu')(x)
  return x


def ResNet(input_shape, n_classes):
    """
    Definition of ResNet
    
    References
    ----------
    https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py
    """
    img_input = layers.Input(shape=input_shape)

    bn_axis = 3

    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [256, 256, 512], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [256, 256, 512], stage=2, block='b')
    x = identity_block(x, 3, [256, 256, 512], stage=2, block='c')

    x = conv_block(x, 3, [512, 512, 1024], stage=3, block='a')
    x = identity_block(x, 3, [512, 512, 1024], stage=3, block='b')
    x = identity_block(x, 3, [512, 512, 1024], stage=3, block='c')
    x = identity_block(x, 3, [512, 512, 1024], stage=3, block='d')
#     x = identity_block(x, 3, [128, 128, 512, 1024], stage=3, block='d')
    
#     x = conv_block(x, 3, [1024, 1024, 2048], stage=4, block='a')
#     x = identity_block(x, 3, [1024, 1024, 2048], stage=4, block='b')
#     x = identity_block(x, 3, [1024, 1024, 2048], stage=4, block='c')
  
    x = layers.Conv2D(n_classes, (1,1), strides=(1, 1),
                    kernel_initializer=glorot_uniform(seed=0),
                    padding='same', name='final_layer')(x)
    img_output = layers.GlobalAveragePooling2D(name='avg_pool')(x)
#     img_output = layers.Dense(n_classes, activation='softmax', name='fc' + str(n_classes))(x)
    img_output = Activation('softmax')(img_output)
    model = Model(inputs=img_input, outputs=img_output, name='resnet')
    return model


model = ResNet((64, 64, 3), 200)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:

from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras import optimizers

clr_triangular = CyclicLR(mode='triangular')

epochs = 80
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
# sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = optimizers.RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])



In [9]:
for epoch in range(epochs):  
  model.fit_generator(train_generator,epochs=1, callbacks=[], samples_per_epoch = 100000, verbose=1)
  if epoch % 10 == 0:
    results = model.evaluate_generator(validation_generator, steps=1) 
    print("epoch "+str(epoch)+" of "+str(epochs))
    print('Accuracy :', (results[1]*100.0))

model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/model.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_"+str(epoch)+"_"+str(results[1])+".h5")
print("Saved model to disk")


Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, callbacks=[], verbose=1, steps_per_epoch=200)`
  


Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


200/200 [==============================] - 407s 2s/step - loss: 4.7170 - acc: 0.0649
epoch 0 of 80
Accuracy : 8.799999952316284
Epoch 1/1
200/200 [==============================] - 393s 2s/step - loss: 4.2405 - acc: 0.1161
Epoch 1/1
200/200 [==============================] - 389s 2s/step - loss: 3.9985 - acc: 0.1466
Epoch 1/1
200/200 [==============================] - 389s 2s/step - loss: 3.8256 - acc: 0.1714
Epoch 1/1
200/200 [==============================] - 389s 2s/step - loss: 3.6820 - acc: 0.1941
Epoch 1/1
200/200 [==============================] - 389s 2s/step - loss: 3.5452 - acc: 0.2176
Epoch 1/1
200/200 [==============================] - 390s 2s/step - loss: 3.4425 - acc: 0.2342
Epoch 1/1
200/200 [==============================] - 390s 2s/step - loss: 3.3395 - acc: 0.2506
Epoch 1/1
200/200 [==============================] - 390s 2s/step - loss: 3.2568 - acc: 0.2651
Epoch 1/1
200/200 [==============================] - 390s 2s/step - loss: 3.1780 - acc: 0.2785
Epoch 1/1
200/200

In [10]:
results = model.evaluate_generator(validation_generator, steps=1) 
print("epoch "+str(epoch))
print('Accuracy :', (results[1]*100.0))

epoch 79
Accuracy : 39.19999897480011


In [0]:
def load_model(model_path, model_json_path):
  from keras.models import Model
  from keras.models import model_from_json
  json_file = open(model_json_path, 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)
  # load weights into new model
  loaded_model.load_weights(model_path)
  print("Loaded model from disk")
  return loaded_model

In [0]:
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_79_0.3959999978542328.h5")

In [0]:
from keras import optimizers

clr_triangular = CyclicLR(mode='triangular')

epochs = 160
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
# sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = optimizers.RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

In [10]:
print('epochs 80 to 160')

for epoch in range(80,160):
  model.fit_generator(train_generator,epochs=1, callbacks=[clr_triangular], samples_per_epoch = 100000, verbose=1)
  if epoch % 10 == 0:
    results = model.evaluate_generator(validation_generator, steps=1) 
    print("epoch "+str(epoch))
    print('Accuracy :', (results[1]*100.0))
  

model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/model.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_"+str(epoch)+"_"+str(results[1])+".h5")
print("Saved model to disk")

epochs 80 to 160
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, callbacks=[<__main__..., verbose=1, steps_per_epoch=200)`
  after removing the cwd from sys.path.


Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


200/200 [==============================] - 418s 2s/step - loss: 4.4549 - acc: 0.1134
epoch 80
Accuracy : 15.000000596046448
Epoch 1/1
200/200 [==============================] - 405s 2s/step - loss: 3.7978 - acc: 0.1742
Epoch 1/1
200/200 [==============================] - 405s 2s/step - loss: 3.5865 - acc: 0.2102
Epoch 1/1
200/200 [==============================] - 405s 2s/step - loss: 3.4762 - acc: 0.2319
Epoch 1/1
200/200 [==============================] - 405s 2s/step - loss: 3.3846 - acc: 0.2503
Epoch 1/1
200/200 [==============================] - 404s 2s/step - loss: 3.2613 - acc: 0.2695
Epoch 1/1
200/200 [==============================] - 403s 2s/step - loss: 3.0493 - acc: 0.2990
Epoch 1/1
200/200 [==============================] - 403s 2s/step - loss: 2.8957 - acc: 0.3276
Epoch 1/1
200/200 [==============================] - 403s 2s/step - loss: 2.7764 - acc: 0.3486
Epoch 1/1
200/200 [==============================] - 403s 2s/step - loss: 2.6782 - acc: 0.3674
Epoch 1/1
200/200 [==

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_159_0.4399999976158142.h5")

from keras import optimizers

clr_triangular = CyclicLR(mode='triangular')

epochs = 160
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
# sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = optimizers.RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print('epochs 160 to 240')

for epoch in range(160,240):
  model.fit_generator(train_generator,epochs=1, callbacks=[clr_triangular], samples_per_epoch = 100000, verbose=1)
  if epoch % 10 == 0:
    results = model.evaluate_generator(validation_generator, steps=1) 
    print("epoch "+str(epoch))
    print('Accuracy :', (results[1]*100.0))
  

model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/model.json", "w") as json_file:
    json_file.write(model_json)

results = model.evaluate_generator(validation_generator, steps=1) 
print("epoch "+str(epoch))
print('Accuracy :', (results[1]*100.0))

model.save_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_"+str(epoch)+"_"+str(results[1])+".h5")
print("Saved model to disk")

epochs 160 to 240


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, callbacks=[<__main__..., verbose=1, steps_per_epoch=200)`


Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


200/200 [==============================] - 422s 2s/step - loss: 0.1000 - acc: 0.9697
epoch 160
Accuracy : 45.60000002384186
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.1133 - acc: 0.9652
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.1395 - acc: 0.9565
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.1697 - acc: 0.9478
Epoch 1/1
200/200 [==============================] - 409s 2s/step - loss: 0.2025 - acc: 0.9375
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.2384 - acc: 0.9263
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.2790 - acc: 0.9148
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.3242 - acc: 0.8997
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.3706 - acc: 0.8880
Epoch 1/1
200/200 [==============================] - 408s 2s/step - loss: 0.4094 - acc: 0.8770
Epoch 1/1
200/200 [==

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model.load_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_239_0.45399999618530273.h5")

from keras import optimizers

clr_triangular = CyclicLR(mode='triangular')

epochs = 160
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
# sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
opt = optimizers.RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print('epochs 240 to 300')

for epoch in range(240,300):
  model.fit_generator(train_generator,epochs=1, callbacks=[clr_triangular], samples_per_epoch = 100000, validation_data = validation_generator , validation_steps = 10000,verbose=1)
  if epoch % 10 == 0:
    results = model.evaluate_generator(validation_generator, steps=1) 
    print("epoch "+str(epoch))
    print('Accuracy :', (results[1]*100.0))
  

model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/model.json", "w") as json_file:
    json_file.write(model_json)

results = model.evaluate_generator(validation_generator, steps=1) 
print("epoch "+str(epoch))
print('Accuracy :', (results[1]*100.0))

model.save_weights("/content/gdrive/My Drive/Colab Notebooks/model_5mar_custom_64x64_"+str(epoch)+"_"+str(results[1])+".h5")
print("Saved model to disk")

epochs 240 to 300


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=1, callbacks=[<__main__..., validation_data=<keras_pre..., validation_steps=10000, verbose=1, steps_per_epoch=200)`


Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


200/200 [==============================] - 7413s 37s/step - loss: 0.0457 - acc: 0.9860 - val_loss: 4.6235 - val_acc: 0.4564
epoch 240
Accuracy : 43.799999356269836
Epoch 1/1
200/200 [==============================] - 7393s 37s/step - loss: 0.0478 - acc: 0.9852 - val_loss: 4.6675 - val_acc: 0.4515
Epoch 1/1
200/200 [==============================] - 7402s 37s/step - loss: 0.0552 - acc: 0.9826 - val_loss: 4.7611 - val_acc: 0.4513
Epoch 1/1
200/200 [==============================] - 7376s 37s/step - loss: 0.0702 - acc: 0.9782 - val_loss: 4.7605 - val_acc: 0.4407
Epoch 1/1
200/200 [==============================] - 7417s 37s/step - loss: 0.0834 - acc: 0.9740 - val_loss: 4.8524 - val_acc: 0.4359
Epoch 1/1
199/200 [============================>.] - ETA: 2s - loss: 0.1080 - acc: 0.9671

On the execution block above you can see 

200/200 [==============================] - 403s 2s/step - loss: 1.5072 - acc: 0.6099

Epoch 1/1

200/200 [==============================] - 403s 2s/step - loss: 1.4070 - acc: 0.6341

Epoch 1/1

200/200 [==============================] - 404s 2s/step - loss: 1.3553 - acc: 0.6473

epoch 100

Accuracy : 49.59999918937683

Epoch 1/1

200/200 [==============================] - 404s 2s/step - loss: 1.3698 - acc: 0.6402

Epoch 1/1

200/200 [==============================] - 403s 2s/step - loss: 1.3801 - acc: 0.6373

Epoch 1/1

200/200 [==============================] - 403s 2s/step - loss: 1.4040 - acc: 0.6294

Epoch 1/1

200/200 [==============================] - 403s 2s/step - loss: 1.4224 - acc: 0.6264

Epoch 1/1

200/200 [==============================] - 403s 2s/step - loss: 1.4543 - acc: 0.6166


This shows that the validation accuracy was at 49.6% at the end of 100th epoch